# ETL 2: Transformación 1 - Limpieza

In [62]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast 

from datetime import datetime, timedelta

pd.options.display.max_columns=None

## Introducción:
Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

**El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes**. 

Para ello:
1. Cargaremos los dos ficheros de datos del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea  
  

In [63]:
clima = pd.read_csv('datos/ETL-1.csv', index_col = 0)
clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 320, 'speed':...",13,15,5,1024,none,0,0,265,2,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 355, 'speed':...",12,15,5,1024,none,0,0,345,2,USA


In [64]:
df = pd.read_csv('datos/Limpieza-4.csv', index_col=0)
df.head(2)

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18.0,Tiger Shark,Jun,Y,M


### Nos quedamos con los países que nos piden:

In [65]:
list_country = ['usa', 'australia', 'south africa', 'new zealand', 'papua new guinea']

In [66]:
df_country = df[df['country'].isin(list_country)]
df_country.head()

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
2,2018,Unprovoked,usa,Walking,15.0,Bull Shark,May,N,M
3,2018,Provoked,australia,Feeding sharks,32.0,Grey Shark,May,N,M
6,2018,Unprovoked,australia,Surfing,60.0,Unspecified,Apr,N,M
8,2018,Unprovoked,south africa,Paddle-skiing,33.0,White Shark,Apr,N,M


In [67]:
df_country['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

### Desempaquetamos la columna 'hr_profile':

In [68]:
clima['rh_profile'] = clima['rh_profile'].apply(ast.literal_eval)

In [69]:
df_rh = clima['rh_profile'].apply(pd.Series)

In [70]:
df_rh.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 4}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 1}","{'layer': '800mb', 'rh': 0}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 0}","{'layer': '650mb', 'rh': 0}","{'layer': '600mb', 'rh': -1}","{'layer': '550mb', 'rh': 1}","{'layer': '500mb', 'rh': 4}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 10}","{'layer': '350mb', 'rh': 11}","{'layer': '300mb', 'rh': 14}","{'layer': '250mb', 'rh': 14}","{'layer': '200mb', 'rh': 5}"
1,"{'layer': '950mb', 'rh': 3}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 1}","{'layer': '800mb', 'rh': 0}","{'layer': '750mb', 'rh': -1}","{'layer': '700mb', 'rh': -1}","{'layer': '650mb', 'rh': -2}","{'layer': '600mb', 'rh': -1}","{'layer': '550mb', 'rh': 2}","{'layer': '500mb', 'rh': 5}","{'layer': '450mb', 'rh': 9}","{'layer': '400mb', 'rh': 10}","{'layer': '350mb', 'rh': 12}","{'layer': '300mb', 'rh': 10}","{'layer': '250mb', 'rh': 13}","{'layer': '200mb', 'rh': 6}"


In [71]:
for i in range(len(df_rh.columns)):
    nombre_rh = "rh_"+ str(df_rh[i].apply(pd.Series)["layer"][0])
    valores_rh = (df_rh[i].apply(pd.Series)["rh"])

    clima.insert(i, nombre_rh, valores_rh)

In [72]:
clima.head()

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,4,3,1,0,0,0,0,-1,1,4,5,10,11,14,14,5,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 320, 'speed':...",13,15,5,1024,none,0,0,265,2,USA
1,3,3,1,0,-1,-1,-2,-1,2,5,9,10,12,10,13,6,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 355, 'speed':...",12,15,5,1024,none,0,0,345,2,USA
2,4,4,4,1,-1,-2,-2,-1,2,7,11,9,10,11,14,10,9,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 2...",13,15,4,1024,none,0,0,260,2,USA
3,4,5,7,6,4,3,0,-1,2,8,11,9,12,14,13,13,12,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 2...",15,10,2,1024,none,0,0,175,2,USA
4,5,7,9,13,9,7,4,1,3,10,11,12,15,15,16,14,15,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",17,6,3,1024,none,0,0,350,2,USA


### Desempaquetamos la columna 'wind_profile':

In [73]:
clima['wind_profile'] = clima['wind_profile'].apply(ast.literal_eval)

In [74]:
df_wind = clima['wind_profile'].apply(pd.Series)
df_wind.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 320, 'speed': 2}","{'layer': '900mb', 'direction': 295, 'speed': 2}","{'layer': '850mb', 'direction': 290, 'speed': 3}","{'layer': '800mb', 'direction': 285, 'speed': 4}","{'layer': '750mb', 'direction': 280, 'speed': 4}","{'layer': '700mb', 'direction': 270, 'speed': 5}","{'layer': '650mb', 'direction': 275, 'speed': 5}","{'layer': '600mb', 'direction': 285, 'speed': 5}","{'layer': '550mb', 'direction': 290, 'speed': 5}","{'layer': '500mb', 'direction': 290, 'speed': 5}","{'layer': '450mb', 'direction': 290, 'speed': 5}","{'layer': '400mb', 'direction': 275, 'speed': 5}","{'layer': '350mb', 'direction': 265, 'speed': 5}","{'layer': '300mb', 'direction': 270, 'speed': 5}","{'layer': '250mb', 'direction': 265, 'speed': 5}","{'layer': '200mb', 'direction': 255, 'speed': 6}"
1,"{'layer': '950mb', 'direction': 355, 'speed': 2}","{'layer': '900mb', 'direction': 310, 'speed': 3}","{'layer': '850mb', 'direction': 305, 'speed': 4}","{'layer': '800mb', 'direction': 285, 'speed': 5}","{'layer': '750mb', 'direction': 275, 'speed': 5}","{'layer': '700mb', 'direction': 270, 'speed': 5}","{'layer': '650mb', 'direction': 270, 'speed': 5}","{'layer': '600mb', 'direction': 275, 'speed': 5}","{'layer': '550mb', 'direction': 280, 'speed': 5}","{'layer': '500mb', 'direction': 280, 'speed': 6}","{'layer': '450mb', 'direction': 275, 'speed': 5}","{'layer': '400mb', 'direction': 255, 'speed': 5}","{'layer': '350mb', 'direction': 240, 'speed': 5}","{'layer': '300mb', 'direction': 250, 'speed': 6}","{'layer': '250mb', 'direction': 255, 'speed': 6}","{'layer': '200mb', 'direction': 255, 'speed': 6}"


In [75]:
for i in range(len(df_wind.columns)): 

    nombre = "wind_direction_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 

    valores = list(df_wind[i].apply(pd.Series)["direction"])

    clima.insert(i, nombre, valores)

In [76]:
clima.head()

,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,320,295,290,285,280,270,275,285,290,290,290,275,265,270,265,255,4,3,1,0,0,0,0,-1,1,4,5,10,11,14,14,5,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 320, 'speed':...",13,15,5,1024,none,0,0,265,2,USA
1,355,310,305,285,275,270,270,275,280,280,275,255,240,250,255,255,3,3,1,0,-1,-1,-2,-1,2,5,9,10,12,10,13,6,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 355, 'speed':...",12,15,5,1024,none,0,0,345,2,USA
2,5,30,10,325,295,285,270,260,255,260,255,240,240,245,255,260,4,4,4,1,-1,-2,-2,-1,2,7,11,9,10,11,14,10,9,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 2...",13,15,4,1024,none,0,0,260,2,USA
3,5,345,330,310,285,270,265,250,250,255,245,240,245,245,250,255,4,5,7,6,4,3,0,-1,2,8,11,9,12,14,13,13,12,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 2...",15,10,2,1024,none,0,0,175,2,USA
4,265,265,270,285,280,270,275,270,255,255,250,255,250,250,250,255,5,7,9,13,9,7,4,1,3,10,11,12,15,15,16,14,15,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",17,6,3,1024,none,0,0,350,2,USA


In [77]:
for i in range(len(df_wind.columns)): 

    nombre = "wind_speed_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 

    valores = list(df_wind[i].apply(pd.Series)["speed"])

    clima.insert(i, nombre, valores)

In [78]:
clima.head()

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,2,2,3,4,4,5,5,5,5,5,5,5,5,5,5,6,320,295,290,285,280,270,275,285,290,290,290,275,265,270,265,255,4,3,1,0,0,0,0,-1,1,4,5,10,11,14,14,5,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 320, 'speed':...",13,15,5,1024,none,0,0,265,2,USA
1,2,3,4,5,5,5,5,5,5,6,5,5,5,6,6,6,355,310,305,285,275,270,270,275,280,280,275,255,240,250,255,255,3,3,1,0,-1,-1,-2,-1,2,5,9,10,12,10,13,6,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 355, 'speed':...",12,15,5,1024,none,0,0,345,2,USA
2,2,2,2,3,3,5,5,5,5,6,6,6,6,6,6,6,5,30,10,325,295,285,270,260,255,260,255,240,240,245,255,260,4,4,4,1,-1,-2,-2,-1,2,7,11,9,10,11,14,10,9,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 2...",13,15,4,1024,none,0,0,260,2,USA
3,2,3,3,3,3,4,5,6,6,6,6,6,6,6,7,7,5,345,330,310,285,270,265,250,250,255,245,240,245,245,250,255,4,5,7,6,4,3,0,-1,2,8,11,9,12,14,13,13,12,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 5, 'speed': 2...",15,10,2,1024,none,0,0,175,2,USA
4,2,3,3,3,4,5,6,6,6,6,6,6,7,7,7,7,265,265,270,285,280,270,275,270,255,255,250,255,250,250,250,255,5,7,9,13,9,7,4,1,3,10,11,12,15,15,16,14,15,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",17,6,3,1024,none,0,0,350,2,USA


In [79]:
clima.columns

Index(['wind_speed_950mb', 'wind_speed_900mb', 'wind_speed_850mb',
       'wind_speed_800mb', 'wind_speed_750mb', 'wind_speed_700mb',
       'wind_speed_650mb', 'wind_speed_600mb', 'wind_speed_550mb',
       'wind_speed_500mb', 'wind_speed_450mb', 'wind_speed_400mb',
       'wind_speed_350mb', 'wind_speed_300mb', 'wind_speed_250mb',
       'wind_speed_200mb', 'wind_direction_950mb', 'wind_direction_900mb',
       'wind_direction_850mb', 'wind_direction_800mb', 'wind_direction_750mb',
       'wind_direction_700mb', 'wind_direction_650mb', 'wind_direction_600mb',
       'wind_direction_550mb', 'wind_direction_500mb', 'wind_direction_450mb',
       'wind_direction_400mb', 'wind_direction_350mb', 'wind_direction_300mb',
       'wind_direction_250mb', 'wind_direction_200mb', 'rh_950mb', 'rh_900mb',
       'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb',
       'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb',
       'rh_250mb', 'rh_200mb', 'timep

### Borramos las dos columnas que hemos desempaquetado, para no tener información repetida.

In [80]:
clima.drop(['rh_profile', 'wind_profile'], axis = 1, inplace=True)

In [81]:
clima.head()

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,2,2,3,4,4,5,5,5,5,5,5,5,5,5,5,6,320,295,290,285,280,270,275,285,290,290,290,275,265,270,265,255,4,3,1,0,0,0,0,-1,1,4,5,10,11,14,14,5,3,9,-9999,-9999,-9999,13,15,5,1024,none,0,0,265,2,USA
1,2,3,4,5,5,5,5,5,5,6,5,5,5,6,6,6,355,310,305,285,275,270,270,275,280,280,275,255,240,250,255,255,3,3,1,0,-1,-1,-2,-1,2,5,9,10,12,10,13,6,6,6,-9999,-9999,-9999,12,15,5,1024,none,0,0,345,2,USA
2,2,2,2,3,3,5,5,5,5,6,6,6,6,6,6,6,5,30,10,325,295,285,270,260,255,260,255,240,240,245,255,260,4,4,4,1,-1,-2,-2,-1,2,7,11,9,10,11,14,10,9,2,-9999,-9999,-9999,13,15,4,1024,none,0,0,260,2,USA
3,2,3,3,3,3,4,5,6,6,6,6,6,6,6,7,7,5,345,330,310,285,270,265,250,250,255,245,240,245,245,250,255,4,5,7,6,4,3,0,-1,2,8,11,9,12,14,13,13,12,3,-9999,-9999,-9999,15,10,2,1024,none,0,0,175,2,USA
4,2,3,3,3,4,5,6,6,6,6,6,6,7,7,7,7,265,265,270,285,280,270,275,270,255,255,250,255,250,250,250,255,5,7,9,13,9,7,4,1,3,10,11,12,15,15,16,14,15,6,-9999,-9999,-9999,17,6,3,1024,none,0,0,350,2,USA


In [82]:
clima['country'] = clima['country'].str.lower()

In [83]:
clima['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [84]:
clima.to_csv('datos/clima-clase.csv')

In [85]:
df_groupby = clima.groupby(["country"])[clima.columns].mean().reset_index()

/var/folders/q5/d6h10gfs1b33l8qy7mdmhspw0000gq/T/ipykernel_10109/747796831.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_groupby = clima.groupby(["country"])[clima.columns].mean().reset_index()


In [86]:
df_groupby.head()

,country,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,australia,3.546875,3.562500,3.500000,3.421875,3.390625,3.515625,3.468750,3.640625,3.828125,3.875000,3.875000,3.812500,3.781250,4.031250,4.750000,5.156250,114.062500,115.468750,114.062500,110.312500,112.812500,115.781250,129.218750,137.031250,144.843750,145.156250,139.609375,141.562500,121.406250,127.500000,147.109375,158.203125,13.531250,11.531250,8.421875,6.265625,5.375000,4.718750,4.093750,3.859375,3.906250,4.828125,4.687500,4.109375,3.187500,2.953125,3.671875,5.375000,97.5,3.906250,-9999.0,-9999.0,-9999.0,25.875000,-3.781250,10.718750,1015.640625,2.703125,0.0,114.843750,3.312500
1,new zealand,3.265625,3.296875,3.343750,3.187500,3.265625,3.312500,3.421875,3.546875,3.625000,3.812500,4.015625,4.234375,4.546875,4.984375,5.828125,6.562500,157.578125,168.437500,185.390625,187.031250,188.437500,186.796875,235.390625,259.453125,263.359375,266.328125,264.921875,257.890625,250.390625,235.859375,243.125000,254.843750,12.390625,13.343750,8.203125,2.203125,1.640625,1.734375,1.484375,0.906250,0.515625,0.609375,0.546875,1.734375,2.781250,3.421875,2.437500,-0.328125,97.5,6.437500,-9999.0,-9999.0,-9999.0,15.062500,10.546875,9.843750,1019.031250,3.890625,0.0,148.515625,2.968750
2,papua new guinea,3.781250,4.187500,4.234375,4.093750,3.359375,2.593750,2.406250,2.765625,3.421875,3.890625,4.296875,4.250000,4.390625,4.531250,5.265625,5.625000,84.140625,82.031250,81.875000,81.406250,78.984375,73.515625,176.796875,229.765625,255.546875,261.484375,259.921875,246.250000,233.281250,216.093750,198.515625,218.984375,13.578125,12.546875,9.406250,6.453125,3.468750,1.781250,1.156250,0.906250,-0.359375,-1.328125,0.531250,0.796875,2.203125,2.718750,3.718750,2.515625,97.5,3.843750,-9999.0,-9999.0,-9999.0,25.953125,-0.531250,11.062500,1010.281250,2.250000,0.0,85.546875,3.234375
3,south africa,2.468750,2.265625,2.156250,2.015625,2.062500,2.125000,2.281250,2.546875,2.562500,3.046875,3.546875,3.718750,3.953125,4.187500,4.609375,5.281250,153.515625,141.328125,148.203125,169.843750,199.609375,204.296875,209.140625,203.437500,216.640625,230.078125,228.359375,233.515625,219.218750,217.421875,218.437500,208.125000,13.078125,9.828125,6.359375,7.031250,8.234375,8.468750,9.921875,7.359375,4.218750,3.203125,1.750000,2.453125,4.750000,5.406250,8.343750,10.140625,97.5,6.375000,-9999.0,-9999.0,-9999.0,23.812500,2.390625,9.906250,1019.406250,1.109375,0.0,139.140625,2.453125
4,usa,2.515625,2.796875,3.093750,3.453125,3.671875,3.968750,4.328125,4.656250,5.000000,5.343750,5.578125,5.796875,6.046875,6.515625,7.140625,6.828125,227.890625,245.390625,284.531250,303.828125,290.156250,293.906250,273.593750,277.343750,276.796875,266.250000,265.703125,259.765625,255.000000,248.828125,256.562500,275.703125,4.187500,4.625000,5.234375,4.984375,4.218750,4.078125,3.296875,2.671875,4.093750,4.890625,4.875000,4.593750,6.000000,7.859375,8.312500,6.468750,97.5,4.921875,-9999.0,-9999.0,-9999.0,12.750000,12.078125,4.296875,1019.296875,0.218750,0.0,238.515625,2.296875


In [87]:
df_completo = df_country.merge(df_groupby, how='outer', on = 'country', right_on= None, left_on=None)

In [88]:
df_groupby.shape

(5, 62)

In [89]:
df_country.shape

(383, 9)

In [90]:
df_completo.shape

(383, 70)

In [91]:
df_completo.head()

,year,type,country,activity,age,species,month,fatal,sex,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F,2.515625,2.796875,3.09375,3.453125,3.671875,3.96875,4.328125,4.65625,5.0,5.34375,5.578125,5.796875,6.046875,6.515625,7.140625,6.828125,227.890625,245.390625,284.53125,303.828125,290.15625,293.90625,273.59375,277.34375,276.796875,266.25,265.703125,259.765625,255.0,248.828125,256.5625,275.703125,4.1875,4.625,5.234375,4.984375,4.21875,4.078125,3.296875,2.671875,4.09375,4.890625,4.875,4.59375,6.0,7.859375,8.3125,6.46875,97.5,4.921875,-9999.0,-9999.0,-9999.0,12.75,12.078125,4.296875,1019.296875,0.21875,0.0,238.515625,2.296875
1,2018,Unprovoked,usa,Walking,15.0,Bull Shark,May,N,M,2.515625,2.796875,3.09375,3.453125,3.671875,3.96875,4.328125,4.65625,5.0,5.34375,5.578125,5.796875,6.046875,6.515625,7.140625,6.828125,227.890625,245.390625,284.53125,303.828125,290.15625,293.90625,273.59375,277.34375,276.796875,266.25,265.703125,259.765625,255.0,248.828125,256.5625,275.703125,4.1875,4.625,5.234375,4.984375,4.21875,4.078125,3.296875,2.671875,4.09375,4.890625,4.875,4.59375,6.0,7.859375,8.3125,6.46875,97.5,4.921875,-9999.0,-9999.0,-9999.0,12.75,12.078125,4.296875,1019.296875,0.21875,0.0,238.515625,2.296875
2,2018,Unprovoked,usa,Stand-Up Paddleboarding,25.0,Tiger Shark,Mar,N,M,2.515625,2.796875,3.09375,3.453125,3.671875,3.96875,4.328125,4.65625,5.0,5.34375,5.578125,5.796875,6.046875,6.515625,7.140625,6.828125,227.890625,245.390625,284.53125,303.828125,290.15625,293.90625,273.59375,277.34375,276.796875,266.25,265.703125,259.765625,255.0,248.828125,256.5625,275.703125,4.1875,4.625,5.234375,4.984375,4.21875,4.078125,3.296875,2.671875,4.09375,4.890625,4.875,4.59375,6.0,7.859375,8.3125,6.46875,97.5,4.921875,-9999.0,-9999.0,-9999.0,12.75,12.078125,4.296875,1019.296875,0.21875,0.0,238.515625,2.296875
3,2017,Unprovoked,usa,Surfing,54.0,Tiger Shark,Dec,N,F,2.515625,2.796875,3.09375,3.453125,3.671875,3.96875,4.328125,4.65625,5.0,5.34375,5.578125,5.796875,6.046875,6.515625,7.140625,6.828125,227.890625,245.390625,284.53125,303.828125,290.15625,293.90625,273.59375,277.34375,276.796875,266.25,265.703125,259.765625,255.0,248.828125,256.5625,275.703125,4.1875,4.625,5.234375,4.984375,4.21875,4.078125,3.296875,2.671875,4.09375,4.890625,4.875,4.59375,6.0,7.859375,8.3125,6.46875,97.5,4.921875,-9999.0,-9999.0,-9999.0,12.75,12.078125,4.296875,1019.296875,0.21875,0.0,238.515625,2.296875
4,2017,Unprovoked,usa,Spearfishing,25.0,White Shark,Nov,N,M,2.515625,2.796875,3.09375,3.453125,3.671875,3.96875,4.328125,4.65625,5.0,5.34375,5.578125,5.796875,6.046875,6.515625,7.140625,6.828125,227.890625,245.390625,284.53125,303.828125,290.15625,293.90625,273.59375,277.34375,276.796875,266.25,265.703125,259.765625,255.0,248.828125,256.5625,275.703125,4.1875,4.625,5.234375,4.984375,4.21875,4.078125,3.296875,2.671875,4.09375,4.890625,4.875,4.59375,6.0,7.859375,8.3125,6.46875,97.5,4.921875,-9999.0,-9999.0,-9999.0,12.75,12.078125,4.296875,1019.296875,0.21875,0.0,238.515625,2.296875



4. Guardad los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [92]:
df_completo.to_csv('datos/ETL-2.csv', index=False)